# Data Preparation

**Authors:** Abderrahmane Salmi, Ricardo Talarico, Lorenzo Allegrini

In [70]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [71]:
tracks_df = pd.read_csv("../datasets/tracks_cleaned.csv")
artists_df = pd.read_csv("../datasets/artists_cleaned.csv")

### Check current features list

In [72]:
# Print all feature names
tracks_original_features = tracks_df.columns.tolist()
print(tracks_original_features)

# Print total number of features
print(f"\nTotal number of features: {len(tracks_original_features)}")

['id_artist', 'name_artist', 'title', 'featured_artists', 'language', 'stats_pageviews', 'swear_IT', 'swear_EN', 'swear_IT_words', 'swear_EN_words', 'year', 'month', 'day', 'n_tokens', 'tokens_per_sent', 'char_per_tok', 'lexical_density', 'avg_token_per_clause', 'bpm', 'centroid', 'rolloff', 'flux', 'flatness', 'spectral_complexity', 'pitch', 'loudness', 'album_name', 'album_release_date', 'album_type', 'disc_number', 'track_number', 'duration_ms', 'explicit', 'popularity', 'lyrics']

Total number of features: 35


In [73]:
# Print all feature names
artists_original_features = artists_df.columns.tolist()
print(artists_original_features)

# Print total number of features
print(f"\nTotal number of features: {len(artists_original_features)}")

['id_author', 'name', 'gender', 'birth_date', 'birth_place', 'nationality', 'description', 'active_start', 'active_end', 'province', 'region', 'country', 'latitude', 'longitude']

Total number of features: 14


### Features Created
- swear_ratio: fraction of words in the songs that are swear words
- aggressiveness: measure of the aggressiveness of the song. Computed based on swear_ratio and musical features like bpm, loudness and flux.
- relative_popularity: ranking of the popularity of the song out of all the songs of the artist
- starting_age: the age at which the artist started making music

In [74]:
tracks_df['swear_ratio'] = (tracks_df['swear_IT'] + tracks_df['swear_EN']) / tracks_df['n_tokens']


from sklearn.preprocessing import StandardScaler

features_for_angriness = ['bpm','centroid','flux','rolloff','loudness','swear_ratio']

scaler = StandardScaler()
scaled_features = scaler.fit_transform(tracks_df[features_for_angriness])
scaled_features = pd.DataFrame(scaled_features, columns=features_for_angriness)


tracks_df['aggressiveness'] = (
    (1/6)*scaled_features['centroid']+
    (1/6)*scaled_features['bpm']+
    (1/6)*scaled_features['flux']+
    (1/6)*scaled_features['swear_ratio']+
    (1/6)*scaled_features['loudness']+
    (1/6)*scaled_features['rolloff']
)
# starting age TODO: check if the code works
artists_df['starting_age'] = math.floor((artists_df['active_start'] - artists_df['birth_date']).days / 365)

# relative popularity
tracks_df['relative_popularity'] = (
    tracks_df
    .groupby('name_artist')['popularity']
    .rank(ascending=False, method='dense')
)


TypeError: unsupported operand type(s) for -: 'str' and 'str'

**artist_age**: Older artists may have more experience, different styles, musical inspirations, lyrics richness, etc.

In [75]:
# Ensure datetime parsing for birth dates
# TODO; we already did this in data understand, delete after check
artists_df['birth_date'] = pd.to_datetime(artists_df['birth_date'], errors='coerce')

# Get the birth year
artists_df['birth_year'] = artists_df['birth_date'].dt.year

# Compute artist age
curr_year = datetime.now().year
artists_df['artist_age'] = curr_year - artists_df['birth_year']

# Optional: handle impossible or missing ages (e.g. negative or NaN)
# artists_df.loc[artists_df['artist_age'] < 10, 'artist_age'] = None  # filter unrealistic ages

# Check that the column exists and contains valid values
artists_df[['id_author', 'birth_year', 'artist_age']].sample(10)

,id_author,birth_year,artist_age
93,ART28717687,1987.0,38.0
53,ART39344115,1994.0,31.0
95,ART85780419,1994.0,31.0
60,ART63613967,1993.0,32.0
57,ART16868977,1992.0,33.0
100,ART15560128,1979.0,46.0
49,ART88792008,1979.0,46.0
78,ART87389753,NaN,NaN
90,ART26418649,1987.0,38.0
34,ART46711784,1989.0,36.0


**artist_age_at_release**: How old the artist was when they released each song, it allows us to detect age-related patterns in lyrics, energy, popularity, etc. Maybe know if successfull songs are mostly made by younger or older artists, which we can be interpreted as hype vs experience.

In [76]:
# Ensure birth_date is datetime
artists_df['birth_date'] = pd.to_datetime(artists_df['birth_date'], errors='coerce')

# Get birth year directly from birth_date
artists_df['birth_year'] = artists_df['birth_date'].dt.year

# Merge birth year from artists_df into tracks_df
tracks_df = tracks_df.merge(
    artists_df[['id_author', 'birth_year']],
    left_on='id_artist',
    right_on='id_author',
    how='left'
)

# Compute artist age at the time of song release
tracks_df['artist_age_at_release'] = tracks_df['year'] - tracks_df['birth_year']

# Optional: filter unrealistic ages
tracks_df.loc[(tracks_df['artist_age_at_release'] < 10) | (tracks_df['artist_age_at_release'] > 100), 'artist_age_at_release'] = None

# Quick check
tracks_df[['title', 'year', 'name_artist', 'birth_year', 'artist_age_at_release']].sample(10)



,title,year,name_artist,birth_year,artist_age_at_release
10862,GUARDA COSA MI FAI FARE,2093.0,Fedez,1989.0,NaN
712,Dovevi essere mia,2013.0,Babaman,1975.0,38.0
9194,Di Tormento Ce N’è Uno,1996.0,Sottotono,NaN,NaN
5517,40 anni,NaN,Dargen D’Amico,NaN,NaN
6414,Come Si Fa,NaN,Entics,1985.0,NaN
9196,Amor de Mi Vida,1999.0,Sottotono,NaN,NaN
4069,Romolo e Remo,2021.0,Tony Effe,1991.0,30.0
5035,Freestyle 7,2018.0,Il Tre,1997.0,21.0
6074,The Pub Song,1915.0,J-Ax,1972.0,NaN
9463,Gabbiano / Moonrock,2018.0,Dani Faiv,1993.0,25.0


### Check all new features

In [77]:
# Find the new features
artists_current_features = artists_df.columns.tolist()
new_artist_features = [col for col in artists_current_features if col not in artists_original_features]

# Print results
print("New Features Added to artists_df:")
for feature in new_artist_features:
    print(f"- {feature}")

print(f"\nTotal new features: {len(new_artist_features)}")

New Features Added to artists_df:
- birth_year
- artist_age

Total new features: 2


In [78]:
# Find the new features
tracks_current_features = tracks_df.columns.tolist()
new_tracks_features = [col for col in tracks_current_features if col not in tracks_original_features]

# Print results
print("New Features Added to artists_df:")
for feature in new_tracks_features:
    print(f"- {feature}")

print(f"\nTotal new features: {len(new_tracks_features)}")

New Features Added to artists_df:
- swear_ratio
- aggressiveness
- id_author
- birth_year
- artist_age_at_release

Total new features: 5
